# O4

## Problem
DONE! - dataanalyse

## Datasæt 
DONE! - dataanalyse

# Valg af ML
* Bruger supervised learning
* Bruger CNN da det er billede data
* Bruger categorical_crossentropy algorith, da vi har flere kategorier eller SparseCategoricalCrossentropy
* Maybe need preprocessed
* Hvordan forholder vi os til over and underfitting
* RandomGridSearch?

## Håndtering af optimering, fordele og ulemper 
* Overfitting/underfitting
*Optimeringsparametre, hyperparameterrummet 

https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/

https://www.tensorflow.org/tutorials/images/classification
```
model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
```

https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d

In [1]:
#make function to fetch dataset

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt


def getDataSet(categories: list, folder: str):
    images = []
    labels = []
    for category in categories:
        folder_path = os.path.join(folder, category)
        label = categories.index(category)  # Assign a numerical label to each category
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (128, 128))  # Resize to a common size
                    img = img / 255.0  # Normalize pixel values to [0, 1]
                    images.append(img)
                    labels.append(label)
                else:
                    print(f"Failed to load image: {img_path}")
    return images, labels


